# Background 
Breast cancer is an uncontrolled growth of cells in the breast. It is the 2nd most common cancer in women and can even also on condition affect man as well. Breast is also the 2nd leading cause death cancer in women after lung cancer. Fine needle aspiration is one commonly used ICT diagnosis methods of breast cancer. By insert the needle in the breast to get the photo of the breast cell. From the photo of begline and malignant cell we can see they are differient in radius, area, roughness, density etc. so we believe those features can classify the cells into benign or malignant by those features. 

# Hypothesis
*Hypothesis*: 
We can find a machine learning method to classify the cell into two kind- benign or malignant, if the cell is benign meaning the people is healthy, if the cell is malignant meaning the people suffer breast cancer. In this way we could detect if the people is under breast cancer or not. And same methodology can be applied to other similar kind cancer detection.

# Methodology and dataset

We collect the raw dataset on Kaggle https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. In the dataset, we have 10 main features 
a) radius 
b) texture 
c) perimeter 
d) area 
e) smoothness 
f) compactness 
g) concavity (severity of concave portions of the contour)
h) concave points (number of concave portions of the contour)
i) symmetry
j) fractal dimension)that can measure the cell
and for each feature we have its mean/se/worst value accordingly, so in total 10*3=30 features for each cell sample. And it is a binary classification problem since the prediction task is to classify the observations in a set of finite labels - begline or malignant.

a.For data exploration
1) We do histogram graphic to see the distribution of malignant cells under each feature, mainly to check if it's normal distribution and also a way to check if we need to delete any features or not 

b.Then we do normalization of our dataset to make columns similar intrinsic influence on analytical model. Here are three ways of normalization 1)simple feature scaling 2)min-max scaling 3)Z-score scaling. We choose option 1)

c. We saperate our dataset 80% for training data and 20% for testing data, and within training data we set 5 groups for cross-validation.

d. We tried differient machine learning algorithms:

1)SVM classifier (in linear kernel, polynomial kernel, RBF kernel)

2)MLP classifier 

3)k-Nearest Neighbour

4)Decision Tree

5)Logistic Regression

e. The results is reported as the accuracy of each classifier, and we calculate the accuracy of the built model using different evaluation metrics: 

1)Jaccard index

2)F1-score

3)LogLoass

In [66]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
df = pd.read_csv('data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


## Data exploration and pre-process


Firstly, see if it's balanced number of M and B type cells.

In [67]:
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

## Data formatting and data preparation

In [68]:
df['diagnosis'].replace(to_replace=['B','M'], value=[0,1],inplace=True) #Convert Categorical features to numerical values
df.head() 

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


# 1.Support Vector Machine (SVM)

Main advantage regarding our dataset:
It is effective in high dimensional spaces

Main disadvantage:
It does not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation
![alt text](https://github.com/YuyingTan/Breast-cancer-detection-Homework/blob/master/svm.JPG?raw=true)

In [89]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((341, 2), (341,), (228, 2), (228,))

## 1.1 SVM with lineal kernel

In [195]:
clf = svm.SVC(kernel='linear', C=0.8).fit(X_train, y_train)
clf

SVC(C=0.8, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [196]:
yhat = clf.predict(X_test)
yhat

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0], dtype=int64)

In [198]:
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Score for each run of the cross validation: ", scores)
# The mean score and the 95% confidence interval of the score estimate are hence given by:
print(">> Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
score = clf.score(X_test, y_test)
print("Mean accuracy on the test data and labels: ", score)

Score for each run of the cross validation:  [0.88043478 0.91304348 0.91208791 0.86666667 0.93333333]
>> Accuracy: 0.90 (+/- 0.05)
Mean accuracy on the test data and labels:  0.8596491228070176


In [199]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
print("SVM_L Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("SVM_L F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )

SVM_L Jaccard index: 0.86
SVM_L F1-score: 0.86


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


## 1.2 SVM with polynomial kernel, degree 3/4/5

In [207]:
clf = svm.SVC(kernel='poly', C=0.8,degree=5).fit(X_train, y_train)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [208]:
yhat = clf.predict(X_test)
yhat

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0], dtype=int64)

In [209]:
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Score for each run of the cross validation: ", scores)
# The mean score and the 95% confidence interval of the score estimate are hence given by:
print(">> Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
score = clf.score(X_test, y_test)
print("Mean accuracy on the test data and labels: ", score)

Score for each run of the cross validation:  [0.83695652 0.84782609 0.76923077 0.77777778 0.82222222]
>> Accuracy: 0.81 (+/- 0.06)
Mean accuracy on the test data and labels:  0.8596491228070176


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: 

In [210]:
print("svm_p Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("svm_p F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )

svm_p Jaccard index: 0.86
svm_p F1-score: 0.85


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


## 1.3 SVM with RBF kernel

In [108]:
clf = svm.SVC(kernel='rbf', C=0.8).fit(X_train, y_train)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [109]:
yhat = clf.predict(X_test)
yhat

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1], dtype=int64)

In [211]:
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Score for each run of the cross validation: ", scores)
# The mean score and the 95% confidence interval of the score estimate are hence given by:
print(">> Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
score = clf.score(X_test, y_test)
print("Mean accuracy on the test data and labels: ", score)

Score for each run of the cross validation:  [0.83695652 0.84782609 0.76923077 0.77777778 0.82222222]
>> Accuracy: 0.81 (+/- 0.06)
Mean accuracy on the test data and labels:  0.8596491228070176


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: 

In [212]:
## print("SVM_R Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("SVM_R F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )
print("svm_R F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )

SVM_R F1-score: 0.85
svm_R F1-score: 0.85


## 1.4SVM with sigmoid kernel

In [160]:
clf = svm.SVC(kernel='sigmoid', C=0.8).fit(X_train, y_train)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [161]:
yhat = clf.predict(X_test)
yhat

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1], dtype=int64)

In [213]:
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Score for each run of the cross validation: ", scores)
# The mean score and the 95% confidence interval of the score estimate are hence given by:
print(">> Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
score = clf.score(X_test, y_test)
print("Mean accuracy on the test data and labels: ", score)

Score for each run of the cross validation:  [0.83695652 0.84782609 0.76923077 0.77777778 0.82222222]
>> Accuracy: 0.81 (+/- 0.06)
Mean accuracy on the test data and labels:  0.8596491228070176


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: 

In [214]:
print("SVM_S Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("SVM_S F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )

SVM_S Jaccard index: 0.86
SVM_S F1-score: 0.85


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


# 2. Multilayer perceptron (MLP)

Multilayer perceptron is a neural network consisting in a minimum of three layers, input layer, output layer and a minimum of one hidden layer.
In MLPs some neurons use a nonlinear activation function that was developed to model the frequency of action potentials, or firing, of biological neurons.
This methop when using a non linear activation funtion between nodes provides classificatio for non linear separated data.
![alt text](https://github.com/YuyingTan/Breast-cancer-detection-Homework/blob/master/MLP.JPG?raw=true)


## 2.1 MLP - adam solver

In [219]:
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(hidden_layer_sizes=(100,),
                           max_iter=10000,
                           activation = 'tanh',
                           solver='adam',
                           random_state=1)
#https://analyticsindiamag.com/a-beginners-guide-to-scikit-learns-mlpclassifier/
#Fitting the training data to the network
mlp=classifier.fit(X_train, y_train)
mlp

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=10000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [220]:
yhat = mlp.predict(X_test)
yhat

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0], dtype=int64)

In [221]:
scores = cross_val_score(mlp, X_train, y_train, cv=5)
print("Score for each run of the cross validation: ", scores)
# The mean score and the 95% confidence interval of the score estimate are hence given by:
print(">> Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
score = mlp.score(X_test, y_test)
print("Mean accuracy on the test data and labels: ", score)

Score for each run of the cross validation:  [0.91304348 0.92391304 0.91208791 0.86666667 0.92222222]
>> Accuracy: 0.91 (+/- 0.04)
Mean accuracy on the test data and labels:  0.8596491228070176


In [222]:
print("MLP_a Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("MLP_a F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )

MLP_a Jaccard index: 0.86
MLP_a F1-score: 0.86


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


## 2.2 MLP - sgd solver

In [227]:
classifier = MLPClassifier(max_iter=10000,
                           activation = 'relu',
                           solver='sgd',
                           learning_rate='adaptive',
                           random_state=1)
mlp=classifier.fit(X_train, y_train)
mlp

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=10000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='sgd', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [228]:
yhat = mlp.predict(X_test)
yhat

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0], dtype=int64)

In [229]:
scores = cross_val_score(mlp, X_train, y_train, cv=5)
print("Score for each run of the cross validation: ", scores)
# The mean score and the 95% confidence interval of the score estimate are hence given by:
print(">> Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
score = mlp.score(X_test, y_test)
print("Mean accuracy on the test data and labels: ", score)

Score for each run of the cross validation:  [0.90217391 0.91304348 0.91208791 0.86666667 0.92222222]
>> Accuracy: 0.90 (+/- 0.04)
Mean accuracy on the test data and labels:  0.8596491228070176


In [230]:
print("MLP_s Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("MLP_s F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )

MLP_s Jaccard index: 0.86
MLP_s F1-score: 0.86


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


# K Nearest Neighbor(KNN)
In a classification problem, the K-Nearest Neighbors algorithm works as follows. 1)pick a value for K. 2)calculate the distance from the new case hold out from each of the cases in the dataset. 3)search for the K-observations in the training data that are nearest to the measurements of the unknown data point. 4) predict the response of the unknown data point using the most popular response value from the K-Nearest Neighbors.
![alt text](https://github.com/YuyingTan/Breast-cancer-detection-Homework/blob/master/KNN.JPG?raw=true)
The K value in the KNN algorithm is important, low K value will result in overfitting but high K value may result in decrease in accuracy, so we choose K equals one and then use the training part for modeling and calculate the accuracy of prediction using all samples in your test set. Repeat this process increasing the K and find the best K for this model.

In [231]:
X = df[['radius_mean','texture_mean']] #defind feature sets, X
X[0:5]
y = df['diagnosis'].values
y[0:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [232]:
X = preprocessing.StandardScaler().fit(X).transform(X) #normalize the data
X[0:5]

array([[ 1.09706398, -2.07333501],
       [ 1.82982061, -0.35363241],
       [ 1.57988811,  0.45618695],
       [-0.76890929,  0.25373211],
       [ 1.75029663, -1.15181643]])

In [233]:
# We split the X into train and test to find the best k
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (455, 2) (455,)
Test set: (114, 2) (114,)


In [234]:
# Modeling
from sklearn.neighbors import KNeighborsClassifier
k = 3
#Train Model and Predict  
kNN_model = KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
kNN_model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [235]:
# just for sanity chaeck
yhat = kNN_model.predict(X_test)
yhat[0:5]

array([0, 0, 1, 1, 1], dtype=int64)

In [236]:
# find the best k
Ks=15
mean_acc=np.zeros((Ks-1))
std_acc=np.zeros((Ks-1))
ConfustionMx=[];
for n in range(1,Ks):
    
    #Train Model and Predict  
    kNN_model = KNeighborsClassifier(n_neighbors=n).fit(X_train,y_train)
    yhat = kNN_model.predict(X_test)
    
    
    mean_acc[n-1]=np.mean(yhat==y_test);
    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
mean_acc

array([0.85087719, 0.9122807 , 0.89473684, 0.92105263, 0.9122807 ,
       0.90350877, 0.9122807 , 0.90350877, 0.89473684, 0.90350877,
       0.89473684, 0.89473684, 0.90350877, 0.90350877])

In [237]:
# Building the model again, using k=4
from sklearn.neighbors import KNeighborsClassifier
k = 4
#Train Model and Predict  
kNN_model = KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
kNN_model

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [238]:
scores = cross_val_score(kNN_model, X_train, y_train, cv=5)
print("Score for each run of the cross validation: ", scores)
# The mean score and the 95% confidence interval of the score estimate are hence given by:
print(">> Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
score = kNN_model.score(X_test, y_test)
print("Mean accuracy on the test data and labels: ", score)

Score for each run of the cross validation:  [0.86956522 0.86956522 0.87912088 0.84444444 0.87777778]
>> Accuracy: 0.87 (+/- 0.02)
Mean accuracy on the test data and labels:  0.9210526315789473


In [239]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
print("KNN Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("KNN F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )

KNN Jaccard index: 0.90
KNN F1-score: 0.90


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


# Decision Tree

Decision trees are built by splitting the training set into distinct nodes, where one node contains all of or most of one category of the data. A decision tree can be constructed by considering the attributes one by one. First, choose an attribute from our dataset. Next, split the data based on the value of the best attribute, then go to each branch and repeat it for the rest of the attributes. After building this tree, you can use it to predict the class of unknown cases.
![alt text](https://github.com/YuyingTan/Breast-cancer-detection-Homework/blob/master/DT.JPG?raw=true)


In [240]:
from sklearn.tree import DecisionTreeClassifier
DT_model = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
DT_model.fit(X_train,y_train)
DT_model

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [241]:
yhat = DT_model.predict(X_test)
yhat

array([0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0], dtype=int64)

In [242]:
scores = cross_val_score(DT_model, X_train, y_train, cv=5)
print("Score for each run of the cross validation: ", scores)
# The mean score and the 95% confidence interval of the score estimate are hence given by:
print(">> Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
score = DT_model.score(X_test, y_test)
print("Mean accuracy on the test data and labels: ", score)

Score for each run of the cross validation:  [0.86956522 0.91304348 0.87912088 0.9        0.91111111]
>> Accuracy: 0.89 (+/- 0.03)
Mean accuracy on the test data and labels:  0.8947368421052632


In [243]:
print("DT Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("DT F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )

DT Jaccard index: 0.89
DT F1-score: 0.90


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


# Logistic Regression

Logistic regression is analogous to linear regression but tries to predict a categorical or discrete target field instead of a numeric one.In logistic regression, we predict a variable which is binary such as yes/no, true/false. It can be used for both binary and multi-class classification. In logistic regression dependent variables should be continuous. If categorical, they should be dummy or indicator coded. 
![alt text](https://github.com/YuyingTan/Breast-cancer-detection-Homework/blob/master/LR.JPG?raw=true)

In [244]:
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(C=0.01).fit(X_train,y_train)
LR_model

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [245]:
yhat = LR_model.predict(X_test)
yhat

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0], dtype=int64)

In [246]:
scores = cross_val_score(LR_model, X_train, y_train, cv=5)
print("Score for each run of the cross validation: ", scores)
# The mean score and the 95% confidence interval of the score estimate are hence given by:
print(">> Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
score = DT_model.score(X_test, y_test)
print("Mean accuracy on the test data and labels: ", score)

Score for each run of the cross validation:  [0.91304348 0.88043478 0.9010989  0.86666667 0.9       ]
>> Accuracy: 0.89 (+/- 0.03)
Mean accuracy on the test data and labels:  0.8947368421052632


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

In [247]:
LR_yhat_prob = LR_model.predict_proba(X_test)
from sklearn.metrics import log_loss
print("LR Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("LR F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )
print("LR LogLoss: %.2f" % log_loss(y_test, LR_yhat_prob))

LR Jaccard index: 0.86
LR F1-score: 0.86
LR LogLoss: 0.45


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


# Model Evaluation
We calculate the accuracy of the built model using different evaluation metrics, results as follows:
![alt text](https://github.com/YuyingTan/Breast-cancer-detection-Homework/blob/master/evaluation.JPG?raw=true)

1)*Jaccard index*: The idea behind this index is that higher the similarity of these two groups the higher the index. It is a general classification model evaluation method to see the similarity of predict value set and actual value set.
![alt text](https://github.com/YuyingTan/Breast-cancer-detection-Homework/blob/master/jaccard%20index.JPG?raw=true)

2)*F1-score*: In statistical analysis of binary classification, the F1 score (also F-score or F-measure) is a measure of a test's accuracy.The F1 score is the harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 (perfect precision and recall) and worst at 0.
![alt text](https://github.com/YuyingTan/Breast-cancer-detection-Homework/blob/master/f1_1.png?raw=true)![alt text](https://github.com/YuyingTan/Breast-cancer-detection-Homework/blob/master/f1_2.png?raw=true)
3)*LogLoss*:Logarithmic loss (related to cross-entropy) measures the performance of a classification model where the prediction input is a probability value between 0 and 1. The goal of our machine learning models is to minimize this value. A perfect model would have a log loss of 0.

# Results 

All algorithms can classify the benign or malignant cell with over 0.85 accuracy. KNN algorithm performes best.

# Extension

1) We can use Grid search method: to solve overfitting or underfitting problem by finding the optimal hyperparameters (c in the SVM, number of hidden layers in Neural Networks etc.).

2) We can find the most influential features by loop delete the features and compare accuracy.

3) We can draw histogram distribution of each features distribution to see if any features need to detele. We draw scatter matrix graphic and radviz plots (It projects a N-dimensional data set into a simple 2D space where the influence of each dimension can be interpreted as a balance between the influence of all dimensions. reference https://cran.r-project.org/web/packages/Radviz/vignettes/single_cell_projections.html). But then we are not advised to do so because it is not white box solution as we cannot know what the x,y axis represent respectively after its transformation in the space. So we are advised to do histogram instead.